In [1]:
from google.colab import drive 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

In [2]:
## log in my google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_path = '/content/drive/MyDrive/Project1-Group 46/train.csv'
test_path = '/content/drive/MyDrive/Project1-Group 46/test.csv'

In [4]:
train_data = pd.read_csv(train_path, header = None)

In [5]:
def DataSplit(data, r1 = 5, r2 = 10, split_size = 0.2):
  #input: loaded data,
  #output: train, validation, test set (np array)
  #explanation: remove the first column (id), feature set is the column from 1 - 960, test set is the column 961

  features = [i for i in range(1,961)]
  labels = [961]
  
  x_train = data[features]
  y_train = data[labels]

  x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = split_size, random_state = r1)
  x_validation, x_test, y_validation, y_test = train_test_split(x_test, y_test, test_size = 0.5, random_state = r2)


  return x_train.values, x_validation.values, x_test.values, y_train.values, y_validation.values, y_test.values


In [6]:
x_train, x_validation, x_test, y_train, y_validation, y_test = DataSplit(train_data)

In [7]:
def To3DFeatures(array):
    new_array = []

    for row in array:
        frame_list = []
        one_frame = []
        joint_coordinate = []
        for index, item in enumerate(row):
          joint_coordinate.append(item)
          if index % 3 == 2:
            one_frame.append(joint_coordinate)
            joint_coordinate = []
          if index % 60 == 59:
              frame_list.append(one_frame)
              one_frame = []
        new_array.append(frame_list)
    
    return new_array

In [8]:
x_train = To3DFeatures(x_train)
x_validation = To3DFeatures(x_validation)
x_test = To3DFeatures(x_test)

x_train = torch.FloatTensor(x_train)
x_validation = torch.FloatTensor(x_validation)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_validation = torch.LongTensor(y_validation)
y_test = torch.LongTensor(y_test)


x_train = np.swapaxes(x_train,1,3)
x_validation = np.swapaxes(x_validation,1,3)
x_test = np.swapaxes(x_test,1,3)

y_train = torch.flatten(y_train)
y_validation = torch.flatten(y_validation)
y_test = torch.flatten(y_test)



train_dataset = TensorDataset(x_train, y_train)
dev_dataset = TensorDataset(x_validation, y_validation)
train_dataloader = DataLoader(train_dataset, batch_size = 100)
dev_dataloader = DataLoader(dev_dataset, batch_size = 100)


In [9]:
class CNN2d_v1(nn.Module):
  def __init__(self, in_channels = 1, num_classes = 10):
    super(CNN2d_v1, self).__init__()

    #20 * 16 * 3
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=3, 
                  out_channels=16, 
                  kernel_size=3, 
                  stride = 1,
                  padding = 1
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2,stride = 2)
    )

    #16 * 10 * 8  
    self.conv2 = nn.Sequential(
         nn.Conv2d(in_channels=16, 
                  out_channels=32, 
                  kernel_size=3, 
                  stride = 1,
                  padding = 1
        ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2,stride = 2) # 40
    )
    #32 * 5 * 4 

    self.fc1 = xnn.Linear(4 * 5 * 32, 50)    


  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.view(-1, 4* 5 * 32)   #Flatten
    x = self.fc1(x)

    return F.log_softmax(x, dim=1)


In [10]:
def train(model, criterion, opti, train_loader, dev_loader, max_eps ):
  best_acc = 0
  st = time.time()

  train_correct = []
  dev_correct = []
  train_losses = []
  dev_losses = []
  for ep in range(max_eps):
    trn_corr = 0
    tst_corr = 0
    total_y = 0
    count = 0
    mean_loss = 0

    for b, (x_train, y_train) in enumerate(train_loader):
      b += 1
      y_pred = model(x_train)
      loss = criterion(y_pred, y_train)

      predicted = torch.max(y_pred.data, 1)[1]
      batch_corr = (predicted == y_train).sum()
      trn_corr += batch_corr

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if b % 30 == 0:
        print(("Iteration {} of epoch {} complete. Loss: {}; Acc =: {}, Time taken (s): {}".format(b, ep, loss.item(), batch_corr/len(y_train), (time.time()-st))))
    

    with torch.no_grad():
      for b,(x_dev, y_dev) in enumerate(dev_loader):
        y_val = model(x_dev)
        predicted = torch.max(y_val.data,1)[1]
        tst_corr += (predicted == y_dev).sum()
        total_y += len(predicted)
        count += 1

        mean_loss += criterion(y_val, y_dev)
    
    acc_dev = tst_corr / total_y
    mean_loss = mean_loss / count

    print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, acc_dev, mean_loss))
    if acc_dev > best_acc:
        print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, acc_dev))
        best_acc = acc_dev
        torch.save(model.state_dict(), 'sstcls_{}.dat'.format(ep))


In [ ]:
model = CNN2d_v1()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
max_eps = 25
train(model, criterion, optimizer, train_dataloader, dev_dataloader, max_eps)


In [12]:
predict_model = CNN2d_v1()
predict_model.load_state_dict(torch.load('sstcls_24.dat'))
with torch.no_grad():
  y_eval = predict_model.forward(x_test)
  loss = criterion(y_eval, y_test)
  prediction = torch.max(y_eval.data,1)[1]

In [13]:
accuracy_score(y_test, prediction)

0.20766773162939298

In [14]:
with torch.no_grad():
  y_eval = model.forward(x_test)
  loss = criterion(y_eval, y_test)
  prediction = torch.max(y_eval.data,1)[1]

In [15]:
accuracy_score(y_test, prediction)

0.16080937167199147